# 5. Obspy

Obspy is the worldwide library for seismic data manipulation in Python. It includes nearly everything you need to work properly at the institute. 

If you want to search for something into Obspy, the website is really well documented [ObsPy Documentation](http://docs.obspy.org/) and you will find for sure the love of your life on it.

Lesson based on [MESS 2014](https://github.com/obspy/mess2014-notebooks/) and [Seismo-Live](https://krischer.github.io/seismo_live_build/tree/index.html)

The core functionality of ObsPy is provided by..

- ..the most important base classes..
  * the **`UTCDateTime`** class handles time information.
  * the **`Stream`**/**`Trace`** classes handle waveform data.
  * the **`Catalog`**/**`Event`**/... classes handle event metadata (modelled after QuakeML).
  * the **`Inventory`**/**`Station`**/**`Response`**/... classes handle station metadata (modelled after FDSN StationXML).


- ..and the associated functions:
  * The **`read`** function. Reads all kinds of waveform file formats. Outputs a **`Stream`** object.
  * The **`read_events`** function. Reads QuakeML (and MCHEDR) files. Outputs a **`Catalog`** object.
  * The **`read_inventory`** function. Reads FDSN StationXML files. Outputs an **`Inventory`** object.


- the most important classes/functions can be imported from main namespace (`from obspy import ...`)
- Unified interface and functionality for handling waveform data regardless of data source
- **`read`**, **`read_events`** and **`read_inventory`** functions access the appropriate file-format submodule/plugin using filetype autodiscovery
- `obspy.core.util` includes some generally useful utility classes/functions (e.g. for geodetic calculations, Flinn-Engdahl regions, ..)
- some convenience command line scripts are also included (e.g. `obspy-plot`, `obspy-print`, `obspy-scan`, ..)


## 5.3. Retrieve Data from Datacenters


ObsPy has clients to directly fetch data via...

- FDSN webservices (IRIS, Geofon/GFZ, USGS, NCEDC, SeisComp3 instances, ...)
- ArcLink (EIDA, ...)
- Earthworm
- SeedLink (near-realtime servers)
- NERIES/NERA/seismicportal.eu
- NEIC
- SeisHub (local seismological database)

This introduction shows how to use the FDSN webservice client. The FDSN webservice definition is by now the default web service implemented by many data centers world wide. Clients for other protocols work similar to the FDSN client.

#### Waveform Data

In [ ]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

client = Client("IRIS")
t = UTCDateTime("2011-03-11T05:46:23")  # Tohoku
st = client.get_waveforms("II", "PFO", "*", "LHZ",
                          t + 10 * 60, t + 30 * 60)
print(st)
st.plot()

In [ ]:
from obspy.clients.seedlink.basic_client import Client

- again, waveform data is returned as a Stream object
- for all custom processing workflows it does not matter if the data originates from a local file or from a web service

#### Event Metadata

The FDSN client can also be used to request event metadata:

In [ ]:
t = UTCDateTime("2011-03-11T05:46:23")  # Tohoku
catalog = client.get_events(starttime=t - 100, endtime=t + 24 * 3600,
                            minmagnitude=9)
print(catalog)

Requests can have a wide range of constraints (see [ObsPy Documentation](http://docs.obspy.org/packages/autogen/obspy.fdsn.client.Client.get_events.html)):

- time range
- geographical (lonlat-box, circular by distance)
- depth range
- magnitude range, type
- contributing agency

#### Station Metadata

Finally, the FDSN client can be used to request station metadata. Stations can be looked up using a wide range of constraints (see [ObsPy documentation](http://docs.obspy.org/packages/autogen/obspy.fdsn.client.Client.get_stations.html)):

 * network/station code
 * time range of operation
 * geographical (lonlat-box, circular by distance)

In [ ]:
event = catalog[0]
origin = event.origins[0]

# Münster
lon = 7.63
lat = 51.96

inventory = client.get_stations(longitude=lon, latitude=lat,
                                maxradius=2.5, level="station")
print(inventory)

The **`level=...`** keyword is used to specify the level of detail in the requested inventory

- `"network"`: only return information on networks matching the criteria
- `"station"`: return information on all matching stations
- `"channel"`: return information on available channels in all stations networks matching the criteria
- `"response"`: include instrument response for all matching channels (large result data size!)

In [ ]:
inventory = client.get_stations(network="OE", station="DAVA",
                                level="station")
print(inventory)

In [ ]:
inventory = client.get_stations(network="OE", station="DAVA",
                                level="channel")
print(inventory)

For waveform requests that include instrument correction, the appropriate instrument response information can be attached to waveforms automatically:     
(Of course, for work on large datasets, the better choice is to download all station information and avoid the internal repeated webservice requests)

In [ ]:
t = UTCDateTime("2011-03-11T05:46:23")  # Tohoku
st = client.get_waveforms("II", "PFO", "*", "LHZ",
                          t + 10 * 60, t + 30 * 60, attach_response=True)
st.plot()

st.remove_response()
st.plot()

All data requested using the FDSN client can be directly saved to file using the **`filename="..."`** option. The data is then stored exactly as it is served by the data center, i.e. without first parsing by ObsPy and outputting by ObsPy.

In [ ]:
client.get_events(starttime=t-100, endtime=t+24*3600, minmagnitude=7,
                  filename="data/requested_events.xml")
client.get_stations(network="OE", station="DAVA", level="station",
                    filename="data/requested_stations.xml")
client.get_waveforms("II", "PFO", "*", "LHZ", t + 10 * 60, t + 30 * 60,
                     filename="data/requested_waveforms.mseed")


In [ ]:
from obspy import read
st = read("data/requested_waveforms.sac")
print(st[0].stats)

#### FDSN Client Exercise

Use the FDSN client to assemble a waveform dataset for on event.

- search for a large earthquake (e.g. by depth or in a region of your choice, use option **`limit=5`** to keep network traffic down)

In [ ]:
from obspy.clients.fdsn import Client



- search for stations to look at waveforms for the event. stations should..
    * be available at the time of the event
    * have a vertical 1 Hz stream ("LHZ", to not overpower our network..)
    * be in a narrow angular distance around the event (e.g. 90-91 degrees)
    * adjust your search so that only a small number of stations (e.g. 3-6) match your search criteria

- for each of these stations download data of the event, e.g. a couple of minutes before to half an hour after the event
- put all data together in one stream (put the `get_waveforms()` call in a try/except/pass block to silently skip stations that actually have no data available)
- print stream info, plot the raw data

In [ ]:
from obspy import Stream


- correct the instrument response for all stations and plot the corrected data

If you have time, assemble and plot another similar dataset (e.g. like before stations at a certain distance from a big event, or use Transportable Array data for a big event, etc.)

#### FDSN Client Correction

Use the FDSN client to assemble a waveform dataset for on event.

- search for a large earthquake (e.g. by depth or in a region of your choice, use option **`limit=5`** to keep network traffic down)

In [ ]:
from obspy.clients.fdsn import Client

client = Client("IRIS")
catalog = client.get_events(minmagnitude=7, mindepth=400, limit=5)
print(catalog)

event = catalog[0]
print(event)

- search for stations to look at waveforms for the event. stations should..
    * be available at the time of the event
    * have a vertical 1 Hz stream ("LHZ", to not overpower our network..)
    * be in a narrow angular distance around the event (e.g. 90-91 degrees)
    * adjust your search so that only a small number of stations (e.g. 3-6) match your search criteria

In [ ]:
event.origins

In [ ]:

origin = event.origins[0]
t = origin.time
lon = origin.longitude
lat = origin.latitude

inventory = client.get_stations(longitude=lon, latitude=lat,
                                minradius=90.5, maxradius=90.8,
                                starttime=t, endtime =t+100,
                                channel="LHZ", matchtimeseries=True)
print(inventory)

- for each of these stations download data of the event, e.g. a couple of minutes before to half an hour after the event
- put all data together in one stream (put the `get_waveforms()` call in a try/except/pass block to silently skip stations that actually have no data available)
- print stream info, plot the raw data

In [ ]:
from obspy import Stream
st = Stream()

for network in inventory:
    for station in network:
        try:
            st += client.get_waveforms(network.code, station.code, "*", "LHZ",
                                       t - 5 * 60, t + 30 * 60, attach_response=True)
        except:
            print('No data')

print(st)
st.plot()

- correct the instrument response for all stations and plot the corrected data

In [ ]:
st.remove_response(water_level=20)
st.plot()